# Aim of the notebook
In this notebook we will convert a SpatialData object to an anndata containing sufficient information to perform spatial RNA velocity

In [1]:
import spatialdata as sd
import scanpy as sc
import spatialdata_plot

/home/sergio/anaconda3/envs/spatialdata/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:21: UserWarning: You are using pyarrow version 12.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


We define the path where the SpatialData object is stored (as zarr)

In [2]:
f='/media/sergio/Meninges/SpatialData_formatted/meninges_h27a'

Next, we read the SpatialData object

In [3]:
sdata=sd.SpatialData.read(f)

INFO     Feature key `feature_name`could be of type `pd.Categorical`. Consider casting it.                         
INFO     Feature key `feature_name`could be of type `pd.Categorical`. Consider casting it.                         


/home/sergio/anaconda3/envs/spatialdata/lib/python3.9/site-packages/anndata/_core/anndata.py:117: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Next we extract both the expression of individual cells (AnnData) and the transcript information

In [12]:
adata=sdata.tables['table']
transcripts=sdata.points['transcripts']

In [19]:
# define maximum distance to nucleus
max_distance_to_nucleus=5
transcripts=transcripts[transcripts['nucleus_distance'].compute()<max_distance_to_nucleus]


KeyboardInterrupt



In [ ]:
import pandas as pd
ex=pd.crosstab(transcripts['feature_name'],transcripts['cell_id'])

/home/sergio/anaconda3/envs/spatialdata/lib/python3.9/site-packages/dask/core.py:127: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))


In [ ]:
ex.head()

In [ ]:
#get cytoplasmic expression
spotsc=transcripts[transcripts['overlaps_nucleus'].compute()==0]
tc=spotsc.loc[:,['feature_name','cell_id']].compute()

In [5]:
#get nuclear expression
spotsn=transcripts[transcripts['overlaps_nucleus'].compute()==1]
tn=spotsn.loc[:,['feature_name','cell_id']].compute()

/home/sergio/anaconda3/envs/spatialdata/lib/python3.9/site-packages/dask/dataframe/core.py:4975: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  meta = self._meta[_extract_meta(key)]


In [ ]:
# create the cellxgene matrix for nuc counts and cytoplasmic
nuc=pd.crosstab(tn['cell_id'],tn['gene'])
cyt=pd.crosstab(tc['cell_id'],tc['gene'])

In [ ]:
#tr2['centroid_distance']=np.sqrt((tr2['x']-tr2['x_cell'])**2+(tr2['y']-tr2['y_cell'])**2)
adata.obs['cell_id']=adata.obs.index
# divide data in nuclei and cytopasm
#adata.var.index=adata.raw.var.index
adata=adata[adata.obs['cell_id'].isin(nuc.index)]
adata=adata[adata.obs['cell_id'].isin(cyt.index)]
nucsort=nuc.loc[adata.obs['cell_id'],adata.var.index]
cytsort=cyt.loc[adata.obs['cell_id'],adata.var.index]
adata.layers['spliced']=np.array(cytsort)
adata.layers['unspliced']=np.array(nucsort)

In [ ]:
adata.write()